In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# custom packages
from ratsimulator import Agent, trajectory_generator, batch_trajectory_generator
from ratsimulator.Environment import Rectangle

import sys
# avoid adding multiple relave paths to sys.path
sys.path.append("../src") if "../src" not in sys.path else None 

from Brain import Brain
from Models import UnitPathIntegrator
from methods import *

### Set parameters and initialise

In [3]:
"""
# Sorscher params
options.save_dir = '/mnt/fs2/bsorsch/grid_cells/models/'
options.n_steps = 100000      # number of training steps
options.batch_size = 200      # number of trajectories per batch
options.sequence_length = 20  # number of steps in trajectory
options.learning_rate = 1e-4  # gradient descent learning rate
options.Np = 512              # number of place cells
options.Ng = 4096             # number of grid cells
options.place_cell_rf = 0.12  # width of place cell center tuning curve (m)
options.surround_scale = 2    # if DoG, ratio of sigma2^2 to sigma1^2
options.RNN_type = 'RNN'      # RNN or LSTM
options.activation = 'relu'   # recurrent nonlinearity
options.weight_decay = 1e-4   # strength of weight decay on recurrent weights
options.DoG = True            # use difference of gaussians tuning curves
options.periodic = False      # trajectories with periodic boundary conditions
options.box_width = 2.2       # width of training environment
options.box_height = 2.2      # height of training environment
"""

params = {}
# Environment params
params['boxsize'] = (2.2, 2.2)
params['origo'] = (0,0)
params['soft_boundary'] = 0.2
# Brain params
params['npcs'] = 512 # as used in Sorscher model
params['pc_sigma'] = 0.12
params['DoG'] = True
# Training data (Agent) params
params['batch_size'] = 64
params['seq_len'] = 1
params['angle0'] = None # random
params['p0'] = None     # random
# Agent/random walk parameters
params['dt'] = 0.02
params['turn_sigma'] = 5.76 * 2
params['b'] = 0.13 * 2 * np.pi
params['mu'] = 0
# Model params
params['Ng'] = 4096
params['Np'] = params['npcs'] # defined for Brain already
params['weight_decay'] = 1e-4
params['lr'] = 1e-4# 1e-3 is default for Adam()
params['nsteps'] = 100 # number of mini batches in an epoch
params['nepochs'] = 10000 # number of epochs

# stuff
params['tag'] = 'default_sorscher'
params['save_model'] = True 
params['save_freq'] = 1

num_workers = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Init Environment
env = Rectangle(boxsize=params['boxsize'], soft_boundary=params['soft_boundary'])
params['environment'] = type(env).__name__
# Init brain
brain = Brain(env, params['npcs'], params['pc_sigma'])
# Init training data
dataset = Dataset(brain=brain, batch_size=params['batch_size'], nsteps=params['nsteps'], \
                  environment=env, seq_len=params['seq_len'], angle0=params['angle0'], \
                  p0=params['p0'], dt=params['dt'], sigma=params['turn_sigma'], b=params['b'], \
                  mu=params['mu'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=params['batch_size'], num_workers=num_workers)
# Init model
model = UnitPathIntegrator(params['Ng'],params['Np'])
model.to(device)
print(model)
# Init optimizer (use custom weight decay, rather than torch optim decay)
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], betas=(0.9, 0.999), \
                             eps=1e-08, weight_decay=0.0, amsgrad=False)

Singular matrix
Singular matrix
UnitPathIntegrator(
  (velocity_encoder): Linear(in_features=2, out_features=4096, bias=False)
  (init_position_encoder): Linear(in_features=512, out_features=4096, bias=False)
  (recurrence): Linear(in_features=4096, out_features=4096, bias=False)
  (decoder): Linear(in_features=4096, out_features=512, bias=False)
)


### Train Model

In [ ]:
checkpoint_path = '../checkpoints/'
loss_history = []

if loaded_model:=True:
    model_name = type(model).__name__
    checkpoints = torch.load(f'{checkpoint_path}{model_name}_{params["tag"]}')
    model.load_state_dict(checkpoints['model_state_dict'])
    optimizer.load_state_dict(checkpoints['optimizer_state_dict'])
    loss_history = checkpoints['loss_history']
    print("Loaded weights")
    
# whether to train
if train:=True:
    model.train(trainloader = dataloader, optimizer = optimizer, weight_decay=params['weight_decay'], \
                nepochs=params['nepochs'], device = device, loaded_model = loaded_model, \
                save_model = params['save_model'], save_freq = params['save_freq'], \
                loss_history = loss_history, tag = params['tag'], params = params)

Epoch=52/10000, loss=5.252010984420776:   1%|▍                                                                                    | 51/10000 [02:08<6:50:13,  2.47s/it]

## OBS OBS! Its actually not markovian (1step).. It needs two steps! Because we do not have POSE we only have POSITION initialization

### Analyse Model

In [ ]:
fig, ax = plt.subplots()
x,y = brain.pcs.T

ax.plot(x, y, "+")
# add standard deviation circles to locations
for i in range(5):
    ax.plot(x[i], y[i], "r+")
    a_circle = plt.Circle((x[i], y[i]), 2 * pc_sigma, fill=False, color=(1, 0, 0, 0.5))
    ax.add_artist(a_circle)

plt.title("Spatial plot of place cell locations")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
# Initialise response maps and create discrete arena
res = 32
res_dx, res_dy = (boxsize[0] - origo[0]) / res, (boxsize[1] - origo[1]) / res
xx, yy = np.meshgrid(np.linspace(origo[0], boxsize[0] - res_dx, res), \
                     np.linspace(origo[1], boxsize[0] - res_dy, res))
arena = np.stack([xx, yy], axis=-1)

num_response_maps = 64
response_maps = np.zeros((num_response_maps, res, res))
idxs = slice(0,64*64,64)

In [ ]:
# Calculate grid cell responses
for n in range(num_samples:=5):
    # sample 'same' positions multiple times - possibly at different head directions
    angle0 = None # None implies random sampled head direction
    for i in range(arena.shape[0]):
        for j in range(arena.shape[1]):
            p0 = np.random.uniform(low=arena[i,j],high=arena[i,j] + np.array([res_dx, res_dy]))
            p0 = p0[None] # p0 needs shape=(1,2)
            # reinitialise pytorch dataset generator
            tg = trajectory_generator(environment=env, seq_len=seq_len, angle0=angle0, p0=p0)
            dataset.tg = tg
            # sample data from generator
            inputs, labels = dataset[0]
            inputs[0] = inputs[0].to(device)
            inputs[1] = inputs[1].to(device)
            grid_cells_response = model.g(inputs)
            response_maps[:,i,j] += grid_cells_response[idxs].detach().cpu().numpy()
response_maps = response_maps / n

In [ ]:
# Plot
ncells = int(np.sqrt(num_response_maps))
fig, ax = plt.subplots(figsize=(10,10),nrows=ncells, ncols=ncells, squeeze=False)

# plot response maps using contourf
for k in range(num_response_maps):
    ax[k // ncells, k % ncells].axis("off")
    # ax[int(k / ncells), k % ncells].set_aspect('equal')
    ax[k // ncells, k % ncells].contourf(
        arena[..., 0], arena[..., 1], response_maps[k], cmap="jet"
    )

In [ ]:
(vel, init_pos), labels = dataset[0]
vel.shape, init_pos.shape, labels.shape

### Strange Sorscher loss point to no path integration learning?

In [ ]:
uniform_vec = np.ones(512) / 512
entropy = lambda x: - np.sum(x * np.log(x))
entropy(uniform_vec)

print(f"{entropy(uniform_vec)=} of uniform vector of same length as sorscher place cell vectors")
print(f"{entropy(init_pos.detach().cpu().numpy())=} of partially trained sorscher model with l2 reg")
print(test)

In [ ]:
(vel, init_pos), labels